# Learnig the representations

In [1]:
# Dependecies
import importlib
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# device agnostic code
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## pasted models
here for convenience

In [3]:
'''
    Representation learning:
    Goal: use unsupervised learning techniques to learn a representation of given data.
    The hope is that this representation will be useful to reduce the amount of data that is needed for training the supervised model for solving the actual task.
    To verify how good the learned representation is, train a supervised model using these representations that predicts the available pretrain labels.

    Methology:
    1. Create a several neural networks that learn to encode the data into a representation.
    2. Train a supervised model on each of the learned representations. The superverised model trained on the different representations should be very shallow (1 or two fully connected layers) and should be trained for a very short time. The goal is to make the performance of the encoders comparable.
'''

# Dependencies
import torch
import torch.nn as nn

'''
    Autoencoder for dimensionality reduction:
    Both encoder and decoder using three linear layers
'''

# for this to make sense the encoding dimension should be significantly smaller than the input dimension
# specifically, the encoding_dim*3 shold be smaller than the input_size
class LinearAutoencoder(nn.Module):
    def __init__(self, input_size, encoding_dim):
        super(LinearAutoencoder, self).__init__()
        # encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_size, encoding_dim*3),
            nn.BatchNorm1d(encoding_dim*3),
            nn.ReLU(),
            nn.Linear(int(encoding_dim*3), int(encoding_dim*2)),
            nn.BatchNorm1d(int(encoding_dim*2)),
            nn.ReLU(),
            nn.Linear(encoding_dim*2, encoding_dim),
        )
        # decoder
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, encoding_dim*2),
            nn.BatchNorm1d(encoding_dim*2),
            nn.ReLU(),
            nn.Linear(encoding_dim*2, encoding_dim*3),
            nn.BatchNorm1d(encoding_dim*3),
            nn.ReLU(),
            nn.Linear(encoding_dim*3, input_size),
            nn.Sigmoid() # the feature values are between 0 and 1
        )

        # fully connected layer for pretrain task
        self.fc = nn.Sequential(
            nn.Linear(encoding_dim, 1)
        )

    def forward(self, x, pretrain = False):
        if pretrain:
            x = self.encoder(x)
            x = self.fc(x)
            x = x.squeeze(1)
        else:
            x = self.encoder(x)
            x = self.decoder(x)
        return x



In [4]:
class ConvAutoencoder(nn.Module):
    def __init__(self, number_filters):
        super(ConvAutoencoder, self).__init__()
        self.number_filters = number_filters
        # encoder
        self.encoder = nn.Sequential(
            nn.Conv1d(1, number_filters*3, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm1d(number_filters*3),
            nn.ReLU(),
            nn.Conv1d(number_filters*3, number_filters*2, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm1d(number_filters*2),
            nn.ReLU(),
            nn.Conv1d(number_filters*2, number_filters, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm1d(number_filters)
        )
        # decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(number_filters, number_filters*2, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm1d(number_filters*2),
            nn.ReLU(),
            nn.ConvTranspose1d(number_filters*2, number_filters*3, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm1d(number_filters*3),
            nn.ReLU(),
            nn.ConvTranspose1d(number_filters*3, 1, kernel_size=3, stride=2, padding=0, output_padding=1), 
            nn.Sigmoid() # the feature values are between 0 and 1
        )
        # fully connected layer for pretrain task
        self.fc = nn.Sequential(
            nn.Linear(124*number_filters, 1)
        )
        
    def forward(self, x, pretrain=False):
        if pretrain:
          x = x.unsqueeze(1)
          x = self.encoder(x)
          x = x.view(-1, 124*self.number_filters)     
          x = self.fc(x)
          x = x.squeeze(1)
        else:
          x = x.unsqueeze(1)
          x = self.encoder(x)
          x = self.decoder(x)
          x = x.squeeze(1)
        return x
       

In [5]:
'''
    Autocoder for dimensionality reduction:
    Using two convolutional/deconvolutional layers and one fully connected layer for both encoder and decoder
'''
class ConvLinearAutoencoder(nn.Module):
    def __init__(self, number_filters, encoding_dim):
        super(ConvLinearAutoencoder, self).__init__()
        self.number_filters = number_filters
        # encoder
        self.encoder = nn.Sequential(
            nn.Conv1d(1, number_filters*2, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm1d(number_filters*2),
            nn.ReLU(),
            nn.Conv1d(number_filters*2, number_filters, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm1d(number_filters),
            nn.ReLU(),
        ) 
        # bottleneck layer
        self.fcencoder = nn.Sequential(
            nn.Linear(249*number_filters, encoding_dim),
            nn.BatchNorm1d(encoding_dim),
        )
        self.fcdecoder = nn.Sequential(
            nn.Linear(encoding_dim, 249*number_filters),
            nn.BatchNorm1d(249*number_filters),
        )

        # decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(number_filters, number_filters*2, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm1d(number_filters*2),
            nn.ReLU(),
            nn.ConvTranspose1d(number_filters*2, 1, kernel_size=3, stride=2, padding=0, output_padding=1),
            nn.Sigmoid() # the feature values are between 0 and 1
        )

        # fully connected layer for pretrain task
        self.fc = nn.Sequential(
            nn.Linear(encoding_dim, 1),
            nn.BatchNorm1d(1),
        )

    def forward(self, x, pretrain = False):
        if pretrain:
            x = x.unsqueeze(1)
            x = self.encoder(x)
            x = x.view(-1, 249*self.number_filters)
            x = self.fcencoder(x)
            x = self.fc(x)
            x = x.squeeze(1)
        else:
            x = x.unsqueeze(1)
            x = self.encoder(x)
            x = x.view(-1, 249*self.number_filters)
            x = self.fcencoder(x)
            x = self.fcdecoder(x)
            x = x.view(-1,self.number_filters, 249)
            x = self.decoder(x)
            x = x.squeeze(1)
        return x


In [6]:
# contractive loss function
def contractive_loss(W, x, recons_x, h, lam=1e-4):
    mse_loss = nn.MSELoss()(recons_x, x)
    
    dh = h * (1 - h) # Derivative of sigmoid
    w_sum = torch.sum(Variable(W)**2, dim=1)
    w_sum = w_sum.unsqueeze(1) # Shape to 2D tensor
    contractive_loss = torch.sum(torch.mm(dh**2, w_sum), 0)
    return mse_loss + contractive_loss.mul_(lam)

## load data

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import shutil
shutil.unpack_archive("drive/MyDrive/SolarEnergyMaterials/task4.zip", "/content/data")
shutil.unpack_archive("data/task4_hr35z9/pretrain_features.csv.zip", "/content/data")
shutil.unpack_archive("data/task4_hr35z9/pretrain_labels.csv.zip", "/content/data")
shutil.unpack_archive("data/task4_hr35z9/train_features.csv.zip", "/content/data")
shutil.unpack_archive("data/task4_hr35z9/train_labels.csv.zip", "/content/data")

In [9]:
def load_pretrain_data(batch_size = 64):
    batch_size = 64

    random.seed(17)
    test_ind = set()

    pre_train_size = 50000

    while len(test_ind) < 10000: 
        test_ind.add(random.randint(0, pre_train_size-1))

    features =[]
    labels = []

    with open("data/pretrain_features.csv", 'r') as f:
        for row in f:
            features.append(row)

    with open("data/pretrain_labels.csv", 'r') as f:
        for row in f:
            labels.append(row)

    # remove header
    features = features[1:]
    labels = labels[1:]

    # first try to note use representation of the molecules, only the extracted features
    features = [list(map(float,row.split(',')[2:])) for row in features]
    labels = [float(row.split(',')[1]) for row in labels]

    train_features = []
    train_labels = []
    test_features = []
    test_labels = []


    for i in range(len(features)):
        if i in test_ind:
            test_features.append(features[i])
            test_labels.append(labels[i])
        else:
            train_features.append(features[i])
            train_labels.append(labels[i])

    # does not seem to make sense to normalize the data since it is very sparse
    # normalize train_features
    # train_features = (train_features - np.mean(train_features, axis=0)) / (np.std(train_features, axis=0)+EPSILON)

    # normalize test_features
    # test_features = (test_features - np.mean(test_features, axis=0)) / (np.std(test_features, axis=0)+EPSILON)

    # convert into tensor dataset
    train_features = torch.tensor(train_features, dtype=torch.float)
    train_labels = torch.tensor(train_labels, dtype=torch.float)
    test_features = torch.tensor(test_features, dtype=torch.float)
    test_labels = torch.tensor(test_labels, dtype=torch.float)

    train_dataset = torch.utils.data.TensorDataset(train_features, train_labels)
    test_dataset = torch.utils.data.TensorDataset(test_features, test_labels) 
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [10]:
train_loader, test_loader = load_pretrain_data(batch_size = 64)

In [23]:
def load_finetune_data(batch_size = 4):
    batch_size = 4

    random.seed(17)
    test_ind = set()

    pre_train_size = 100
    while len(test_ind) < 50: 
        test_ind.add(random.randint(0, pre_train_size-1))

    features =[]
    labels = []

    with open("data/train_features.csv", 'r') as f:
        for row in f:
            features.append(row)

    with open("data/train_labels.csv", 'r') as f:
        for row in f:
            labels.append(row)

    # remove header
    features = features[1:]
    labels = labels[1:]

    # first try to note use representation of the molecules, only the extracted features
    features = [list(map(float,row.split(',')[2:])) for row in features]
    labels = [float(row.split(',')[1]) for row in labels]

    train_features = []
    train_labels = []
    test_features = []
    test_labels = []


    for i in range(len(features)):
        if i in test_ind:
            test_features.append(features[i])
            test_labels.append(labels[i])
        else:
            train_features.append(features[i])
            train_labels.append(labels[i])

    # does not seem to make sense to normalize the data since it is very sparse
    # normalize train_features
    # train_features = (train_features - np.mean(train_features, axis=0)) / (np.std(train_features, axis=0)+EPSILON)

    # normalize test_features
    # test_features = (test_features - np.mean(test_features, axis=0)) / (np.std(test_features, axis=0)+EPSILON)

    # convert into tensor dataset
    train_features = torch.tensor(train_features, dtype=torch.float)
    train_labels = torch.tensor(train_labels, dtype=torch.float)
    test_features = torch.tensor(test_features, dtype=torch.float)
    test_labels = torch.tensor(test_labels, dtype=torch.float)

    train_dataset = torch.utils.data.TensorDataset(train_features, train_labels)
    test_dataset = torch.utils.data.TensorDataset(test_features, test_labels) 
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [24]:
finetune_train_loader, finetune_test_loader = load_finetune_data(batch_size=4)

## Train/Test loop

In [11]:
# train loop:
def train_encoder(model, dataloader, epochs, pretrain=False):
    if pretrain:
      freeze_weights(model.encoder)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()

    for epoch in range(epochs):
        epoch_loss=0
        for batch, (X, y) in enumerate(dataloader):
          if pretrain:
            optimizer.zero_grad()
            y_pred = model(X, pretrain=True)
            loss = loss_fn(y, y_pred)
            epoch_loss+=loss.item()
            loss.backward()  
            optimizer.step()
          else:
            optimizer.zero_grad()
            X_pred = model(X)
            loss = loss_fn(X_pred, X)
            epoch_loss+=loss.item()
            loss.backward()  
            optimizer.step()

        print('average loss per batch in epoch [{}/{}], Loss: {:.6f}'.format(epoch+1, epochs, epoch_loss/len(dataloader)))

In [12]:
# test loop
def test_model(model, data_loader):
  loss_fn = nn.MSELoss() 
  model.to(device)
  Y = torch.tensor([]).to(device)
  Y_pred = torch.tensor([]).to(device)
  with torch.no_grad():
    for batch, (X,y) in enumerate(data_loader):
      X = X.to(device)
      y = y.to(device)
      y_pred = model(X, pretrain=True)
      Y = torch.cat((Y, y))
      Y_pred = torch.cat((Y_pred, y_pred))
    loss = torch.sqrt(loss_fn(y_pred, y))
    print(f"average batch loss: {loss.item()}")

In [13]:
def freeze_weights(model):
    for param in model.parameters():
        param.requires_grad = False


## Linear Autoencoder:

In [14]:
LinearEncoder = LinearAutoencoder(1000, 128)
train_encoder(LinearEncoder, train_loader, 10)

average loss per batch in epoch [1/10], Loss: 0.017232
average loss per batch in epoch [2/10], Loss: 0.007259
average loss per batch in epoch [3/10], Loss: 0.005316
average loss per batch in epoch [4/10], Loss: 0.004196
average loss per batch in epoch [5/10], Loss: 0.003411
average loss per batch in epoch [6/10], Loss: 0.002907
average loss per batch in epoch [7/10], Loss: 0.002534
average loss per batch in epoch [8/10], Loss: 0.002269
average loss per batch in epoch [9/10], Loss: 0.002070
average loss per batch in epoch [10/10], Loss: 0.001909


In [15]:
torch.save(LinearEncoder.state_dict(), 'drive/MyDrive/SolarEnergyMaterials/PretrainedModels/LinearEncoder.pth')

## Convolutional Autoencoder:

In [16]:
ConvEncoder = ConvAutoencoder(4)
test = next(iter(train_loader))
train_encoder(ConvEncoder, train_loader, 10)

average loss per batch in epoch [1/10], Loss: 0.033020
average loss per batch in epoch [2/10], Loss: 0.006542
average loss per batch in epoch [3/10], Loss: 0.001901
average loss per batch in epoch [4/10], Loss: 0.000984
average loss per batch in epoch [5/10], Loss: 0.000596
average loss per batch in epoch [6/10], Loss: 0.000337
average loss per batch in epoch [7/10], Loss: 0.000266
average loss per batch in epoch [8/10], Loss: 0.000220
average loss per batch in epoch [9/10], Loss: 0.000199
average loss per batch in epoch [10/10], Loss: 0.000183


In [17]:
torch.save(ConvEncoder.state_dict(), 'drive/MyDrive/SolarEnergyMaterials/PretrainedModels/ConvEncoder.pth')

## Convolutional Autoencoder with Linear Layer:

In [18]:
ConvLinearEncoder = ConvLinearAutoencoder(6, 90) # almost no compression if product close to 1000
train_encoder(ConvLinearEncoder, train_loader, 10)

average loss per batch in epoch [1/10], Loss: 0.066300
average loss per batch in epoch [2/10], Loss: 0.008494
average loss per batch in epoch [3/10], Loss: 0.004696
average loss per batch in epoch [4/10], Loss: 0.003250
average loss per batch in epoch [5/10], Loss: 0.002525
average loss per batch in epoch [6/10], Loss: 0.002059
average loss per batch in epoch [7/10], Loss: 0.001743
average loss per batch in epoch [8/10], Loss: 0.001523
average loss per batch in epoch [9/10], Loss: 0.001357
average loss per batch in epoch [10/10], Loss: 0.001229


In [19]:
torch.save(ConvEncoder.state_dict(), 'drive/MyDrive/SolarEnergyMaterials/PretrainedModels/ConvEncoder.pth')

## Evaluating the representations
With the pretrain data

### Linear Model


In [20]:
train_encoder(LinearEncoder, train_loader, 10, pretrain=True)
test_model(LinearEncoder, train_loader)
print('---')
test_model(LinearEncoder, test_loader)

average loss per batch in epoch [1/10], Loss: 0.461614
average loss per batch in epoch [2/10], Loss: 0.087172
average loss per batch in epoch [3/10], Loss: 0.075728
average loss per batch in epoch [4/10], Loss: 0.067485
average loss per batch in epoch [5/10], Loss: 0.061753
average loss per batch in epoch [6/10], Loss: 0.056698
average loss per batch in epoch [7/10], Loss: 0.054565
average loss per batch in epoch [8/10], Loss: 0.052156
average loss per batch in epoch [9/10], Loss: 0.050331
average loss per batch in epoch [10/10], Loss: 0.049903
average batch loss: 0.2667633295059204
---
average batch loss: 0.1762481927871704


### Conv Model


In [21]:
train_encoder(ConvEncoder, train_loader, 10, pretrain=True)
test_model(ConvEncoder, train_loader)
print('---')
test_model(ConvEncoder, test_loader)

average loss per batch in epoch [1/10], Loss: 0.333644
average loss per batch in epoch [2/10], Loss: 0.071668
average loss per batch in epoch [3/10], Loss: 0.055855
average loss per batch in epoch [4/10], Loss: 0.050241
average loss per batch in epoch [5/10], Loss: 0.047170
average loss per batch in epoch [6/10], Loss: 0.045823
average loss per batch in epoch [7/10], Loss: 0.043739
average loss per batch in epoch [8/10], Loss: 0.042354
average loss per batch in epoch [9/10], Loss: 0.041588
average loss per batch in epoch [10/10], Loss: 0.039851
average batch loss: 0.1723169982433319
---
average batch loss: 0.14731952548027039


### Conv Linear Model

In [22]:
train_encoder(ConvLinearEncoder, train_loader, 10, pretrain=True) # why is the initial loss so high compared to the others, might suggest that the representations are not working?
test_model(ConvLinearEncoder, train_loader)
print('---')
test_model(ConvLinearEncoder, test_loader) 

average loss per batch in epoch [1/10], Loss: 9.193672
average loss per batch in epoch [2/10], Loss: 5.929078
average loss per batch in epoch [3/10], Loss: 3.673273
average loss per batch in epoch [4/10], Loss: 2.110967
average loss per batch in epoch [5/10], Loss: 1.092343
average loss per batch in epoch [6/10], Loss: 0.490752
average loss per batch in epoch [7/10], Loss: 0.185499
average loss per batch in epoch [8/10], Loss: 0.061543
average loss per batch in epoch [9/10], Loss: 0.025520
average loss per batch in epoch [10/10], Loss: 0.018551
average batch loss: 0.13092286884784698
---
average batch loss: 0.10595816373825073


## Train model on task
Goal is to test the hypothesis that good representations make it easier to train models on a wide range of differnent tasks. More precisely:
If a model create encodings that lead to relatively good performance on the pretraining problem, then we expect it to score similar (relatively) on the actual problem. For starters we will just use one linear layer to train ontop of the encoders.


#### Linear Encoder

In [51]:
# using not pretrained model
# using control to check whether it makes a diffrence to learn the representations at all
controlLinearEncoder = LinearAutoencoder(1000, 128)
expLinearEncoder = LinearAutoencoder(1000, 128)
train_encoder(expLinearEncoder, train_loader, 10)
torch.save(expLinearEncoder.state_dict(), 'drive/MyDrive/SolarEnergyMaterials/PretrainedModels/expLinearEncoder.pth')
torch.save(controlLinearEncoder.state_dict(), 'drive/MyDrive/SolarEnergyMaterials/PretrainedModels/controlLinearEncoder.pth')

average loss per batch in epoch [1/10], Loss: 0.017215
average loss per batch in epoch [2/10], Loss: 0.007233
average loss per batch in epoch [3/10], Loss: 0.005292
average loss per batch in epoch [4/10], Loss: 0.004201
average loss per batch in epoch [5/10], Loss: 0.003455
average loss per batch in epoch [6/10], Loss: 0.002937
average loss per batch in epoch [7/10], Loss: 0.002558
average loss per batch in epoch [8/10], Loss: 0.002277
average loss per batch in epoch [9/10], Loss: 0.002084
average loss per batch in epoch [10/10], Loss: 0.001910


In [58]:
# train on actual problem
controlLinearEncoder = LinearAutoencoder(1000, 128)
expLinearEncoder = LinearAutoencoder(1000, 128)
controlLinearEncoder.load_state_dict(torch.load('drive/MyDrive/SolarEnergyMaterials/PretrainedModels/controlLinearEncoder.pth'),strict=True)
expLinearEncoder.load_state_dict(torch.load('drive/MyDrive/SolarEnergyMaterials/PretrainedModels/expLinearEncoder.pth'),strict=True)

train_encoder(expLinearEncoder, finetune_train_loader, epochs = 10, pretrain = True)
print('---')
train_encoder(controlLinearEncoder, finetune_train_loader, epochs=10, pretrain = True)
print('------')
test_model(expLinearEncoder, finetune_train_loader)
test_model(expLinearEncoder, finetune_test_loader)
print('---')
test_model(controlLinearEncoder, finetune_train_loader)
test_model(controlLinearEncoder, finetune_test_loader)

average loss per batch in epoch [1/10], Loss: 4.138478
average loss per batch in epoch [2/10], Loss: 0.983916
average loss per batch in epoch [3/10], Loss: 0.793954
average loss per batch in epoch [4/10], Loss: 0.574076
average loss per batch in epoch [5/10], Loss: 0.468468
average loss per batch in epoch [6/10], Loss: 0.415740
average loss per batch in epoch [7/10], Loss: 0.420678
average loss per batch in epoch [8/10], Loss: 0.486699
average loss per batch in epoch [9/10], Loss: 0.331414
average loss per batch in epoch [10/10], Loss: 0.313677
---
average loss per batch in epoch [1/10], Loss: 2.786418
average loss per batch in epoch [2/10], Loss: 1.837831
average loss per batch in epoch [3/10], Loss: 1.146003
average loss per batch in epoch [4/10], Loss: 0.695022
average loss per batch in epoch [5/10], Loss: 0.418397
average loss per batch in epoch [6/10], Loss: 0.287914
average loss per batch in epoch [7/10], Loss: 0.169316
average loss per batch in epoch [8/10], Loss: 0.218471
avera

#### Convolutioal Encoder

In [39]:
# using not pretrained model
# using control to check whether it makes a diffrence to learn the representations at all
controlConvEncoder = ConvAutoencoder(4)
expConvEncoder = ConvAutoencoder(4)
train_encoder(expConvEncoder, train_loader, 10)
torch.save(expConvEncoder.state_dict(), 'drive/MyDrive/SolarEnergyMaterials/PretrainedModels/expConvEncoder.pth')
torch.save(controlConvEncoder.state_dict(), 'drive/MyDrive/SolarEnergyMaterials/PretrainedModels/controlConvEncoder.pth')

average loss per batch in epoch [1/10], Loss: 0.071669
average loss per batch in epoch [2/10], Loss: 0.003988
average loss per batch in epoch [3/10], Loss: 0.001851
average loss per batch in epoch [4/10], Loss: 0.001323
average loss per batch in epoch [5/10], Loss: 0.001168
average loss per batch in epoch [6/10], Loss: 0.001089
average loss per batch in epoch [7/10], Loss: 0.000840
average loss per batch in epoch [8/10], Loss: 0.000657
average loss per batch in epoch [9/10], Loss: 0.000560
average loss per batch in epoch [10/10], Loss: 0.000472


In [48]:
# train on actual problem
controlConvEncoder = ConvAutoencoder(4)
expConvEncoder = ConvAutoencoder(4)
controlConvEncoder.load_state_dict(torch.load('drive/MyDrive/SolarEnergyMaterials/PretrainedModels/controlConvEncoder.pth'),strict=True)
expConvEncoder.load_state_dict(torch.load('drive/MyDrive/SolarEnergyMaterials/PretrainedModels/expConvEncoder.pth'),strict=True)
train_encoder(expConvEncoder, finetune_train_loader, epochs = 10, pretrain = True)
print('---')
train_encoder(controlConvEncoder, finetune_train_loader, epochs=10, pretrain = True)
print('------')
test_model(expConvEncoder, finetune_train_loader)
test_model(expConvEncoder, finetune_test_loader)
print('---')
test_model(controlConvEncoder, finetune_train_loader)
test_model(controlConvEncoder, finetune_test_loader)

average loss per batch in epoch [1/10], Loss: 1.764649
average loss per batch in epoch [2/10], Loss: 0.524801
average loss per batch in epoch [3/10], Loss: 0.369193
average loss per batch in epoch [4/10], Loss: 0.179534
average loss per batch in epoch [5/10], Loss: 0.129048
average loss per batch in epoch [6/10], Loss: 0.078349
average loss per batch in epoch [7/10], Loss: 0.047434
average loss per batch in epoch [8/10], Loss: 0.038616
average loss per batch in epoch [9/10], Loss: 0.030425
average loss per batch in epoch [10/10], Loss: 0.024886
---
average loss per batch in epoch [1/10], Loss: 3.798221
average loss per batch in epoch [2/10], Loss: 0.387137
average loss per batch in epoch [3/10], Loss: 0.294806
average loss per batch in epoch [4/10], Loss: 0.173692
average loss per batch in epoch [5/10], Loss: 0.118407
average loss per batch in epoch [6/10], Loss: 0.093576
average loss per batch in epoch [7/10], Loss: 0.072977
average loss per batch in epoch [8/10], Loss: 0.043856
avera

#### Convolutional Linear Encoder


In [52]:
# using not pretrained model
# using control to check whether it makes a diffrence to learn the representations at all
controlConvLinearEncoder = ConvLinearAutoencoder(6,90)
expConvLinearEncoder = ConvLinearAutoencoder(6,90)
train_encoder(expConvLinearEncoder, train_loader, 10)
torch.save(expConvLinearEncoder.state_dict(), 'drive/MyDrive/SolarEnergyMaterials/PretrainedModels/expConvLinearEncoder.pth')
torch.save(controlConvLinearEncoder.state_dict(), 'drive/MyDrive/SolarEnergyMaterials/PretrainedModels/controlConvLinearEncoder.pth')

average loss per batch in epoch [1/10], Loss: 0.073812
average loss per batch in epoch [2/10], Loss: 0.008143
average loss per batch in epoch [3/10], Loss: 0.004759
average loss per batch in epoch [4/10], Loss: 0.003401
average loss per batch in epoch [5/10], Loss: 0.002687
average loss per batch in epoch [6/10], Loss: 0.002248
average loss per batch in epoch [7/10], Loss: 0.001942
average loss per batch in epoch [8/10], Loss: 0.001711
average loss per batch in epoch [9/10], Loss: 0.001558
average loss per batch in epoch [10/10], Loss: 0.001409


In [53]:
# train on actual problem
controlConvLinearEncoder = ConvLinearAutoencoder(6,90)
expConvLinearEncoder = ConvLinearAutoencoder(6,90)
controlConvLinearEncoder.load_state_dict(torch.load('drive/MyDrive/SolarEnergyMaterials/PretrainedModels/controlConvLinearEncoder.pth'),strict=True)
expConvLinearEncoder.load_state_dict(torch.load('drive/MyDrive/SolarEnergyMaterials/PretrainedModels/expConvLinearEncoder.pth'),strict=True)
train_encoder(expConvLinearEncoder, finetune_train_loader, epochs = 10, pretrain = True)
print('---')
train_encoder(controlConvLinearEncoder, finetune_train_loader, epochs=10, pretrain = True)
print('------')
test_model(expConvLinearEncoder, finetune_train_loader)
test_model(expConvLinearEncoder, finetune_test_loader)
print('---')
test_model(controlConvLinearEncoder, finetune_train_loader)
test_model(controlConvLinearEncoder, finetune_test_loader)

average loss per batch in epoch [1/10], Loss: 4.249253
average loss per batch in epoch [2/10], Loss: 4.008540
average loss per batch in epoch [3/10], Loss: 3.882495
average loss per batch in epoch [4/10], Loss: 3.822384
average loss per batch in epoch [5/10], Loss: 3.613687
average loss per batch in epoch [6/10], Loss: 3.682663
average loss per batch in epoch [7/10], Loss: 3.607144
average loss per batch in epoch [8/10], Loss: 3.503795
average loss per batch in epoch [9/10], Loss: 3.383416
average loss per batch in epoch [10/10], Loss: 3.453778
---
average loss per batch in epoch [1/10], Loss: 4.414052
average loss per batch in epoch [2/10], Loss: 4.039785
average loss per batch in epoch [3/10], Loss: 3.782329
average loss per batch in epoch [4/10], Loss: 3.814222
average loss per batch in epoch [5/10], Loss: 3.782018
average loss per batch in epoch [6/10], Loss: 3.628115
average loss per batch in epoch [7/10], Loss: 3.702206
average loss per batch in epoch [8/10], Loss: 3.516008
avera